In [21]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pickle

In [2]:
data = pd.read_excel('OnlineRetail (1).xlsx')

In [3]:
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [4]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data['Year'] = data['InvoiceDate'].dt.year
data['Month'] = data['InvoiceDate'].dt.month

In [16]:
def get_recommendations(item_name, threshold=0.2, top_n=5, item_correlation=None):
    similar_scores = item_correlation[item_name].sort_values(ascending=False)
    similar_items = similar_scores[similar_scores > threshold].index.tolist()
    similar_items.remove(item_name)
    return similar_items[:top_n]

In [5]:
def generate_recommendations(data, threshold=0.2, top_n=3):
    global_pivot = data.pivot_table(index='CustomerID', columns='Description', values='Quantity', aggfunc='sum', fill_value=0)
    global_item_correlation = global_pivot.corr(method='pearson', min_periods=5)

    recommendations_by_country = {}
    for country in data['Country'].unique():
        country_data = data[data['Country'] == country]
        country_pivot = country_data.pivot_table(index='CustomerID', columns='Description', values='Quantity', aggfunc='sum', fill_value=0)
        country_item_correlation = country_pivot.corr(method='pearson', min_periods=5)
        recommendations_by_month = {}

        for year_month, month_data in country_data.groupby(['Year', 'Month']):
            month_pivot = month_data.pivot_table(index='CustomerID', columns='Description', values='Quantity', aggfunc='sum', fill_value=0)
            month_item_correlation = month_pivot.corr(method='pearson', min_periods=5)

            recommendations = {}
            for item_name in data['Description'].unique():
                if item_name in month_item_correlation.columns:
                    similar_scores = month_item_correlation[item_name].sort_values(ascending=False)
                    similar_items = similar_scores[similar_scores > threshold].index.tolist()
                    if item_name in similar_items:
                        similar_items.remove(item_name)
                    recommendations[item_name] = similar_items[:top_n]

            recommendations_by_month[year_month] = recommendations
        recommendations_by_country[country] = recommendations_by_month
    
    return global_pivot, global_item_correlation, recommendations_by_country

In [19]:
global_pivot

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12347.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12348.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12349.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12350.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18281.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18282.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
global_pivot, global_item_correlation, recommendations_by_country = generate_recommendations(data)

In [17]:
item_name = "WHITE HANGING HEART T-LIGHT HOLDER"
global_recommendations = get_recommendations(item_name, threshold=0.2, top_n=5, item_correlation=global_item_correlation)
print(f"Global Recommendations for '{item_name}':")
print(global_recommendations)

Global Recommendations for 'WHITE HANGING HEART T-LIGHT HOLDER':
['RED HANGING HEART T-LIGHT HOLDER', 'WASHROOM METAL SIGN', 'LAUNDRY 15C METAL SIGN', 'GREEN VINTAGE SPOT BEAKER', 'BLUE VINTAGE SPOT BEAKER']


In [27]:
# Load recommendations_by_country from the pickle file
with open('recommendations_by_country.pkl', 'rb') as f:
    recommendations_by_country = pickle.load(f)

# Now you can retrieve recommendations for a specific country
country_name = "United Kingdom"
item_name = "WHITE HANGING HEART T-LIGHT HOLDER"

# Retrieve country-wise recommendations
country_recommendations = recommendations_by_country[country_name]

# Iterate through months and retrieve recommendations for the specified item
for year_month, month_recommendations in country_recommendations.items():
    if item_name in month_recommendations:
        item_recommendations = month_recommendations[item_name]
        print(f"Month-wise Recommendations for '{item_name}' in '{country_name}' for {year_month}:")
        print(item_recommendations)
    else:
        print(f"No recommendations found for '{item_name}' in '{country_name}' for {year_month}")

Month-wise Recommendations for 'WHITE HANGING HEART T-LIGHT HOLDER' in 'United Kingdom' for (2010, 12):
['STRAWBERRY CERAMIC TRINKET BOX', 'RED WOOLLY HOTTIE WHITE HEART.', 'PINK MARSHMALLOW SCARF KNITTING KIT']
Month-wise Recommendations for 'WHITE HANGING HEART T-LIGHT HOLDER' in 'United Kingdom' for (2011, 1):
['GIN + TONIC DIET METAL SIGN', 'DOORMAT FAIRY CAKE', 'TEA TIME TEA TOWELS']
Month-wise Recommendations for 'WHITE HANGING HEART T-LIGHT HOLDER' in 'United Kingdom' for (2011, 2):
['COLOUR GLASS. STAR T-LIGHT HOLDER', 'STRAWBERRY RAFFIA FOOD COVER', 'PARTY BUNTING']
Month-wise Recommendations for 'WHITE HANGING HEART T-LIGHT HOLDER' in 'United Kingdom' for (2011, 3):
['ROSE SCENT CANDLE IN JEWELLED BOX', '6 CHOCOLATE LOVE HEART T-LIGHTS', 'TOAST ITS - BEST MUM']
Month-wise Recommendations for 'WHITE HANGING HEART T-LIGHT HOLDER' in 'United Kingdom' for (2011, 4):
['PARTY FOOD SHOPPER BAG', 'HOT BATHS METAL SIGN', 'BLACK SMALL GLASS CAKE STAND']
Month-wise Recommendations for '

In [26]:
year_month = (2011, 1)  # Change this to the desired year and month
month_recommendations = recommendations_by_country[country_name][year_month][item_name]
print(f"Month-wise Recommendations for '{item_name}' in '{country_name}' for {year_month}:")
print(month_recommendations)

Month-wise Recommendations for '10 COLOUR SPACEBOY PEN' in 'United Kingdom' for (2011, 1):
['HOT WATER BOTTLE BABUSHKA', 'SET OF 6 T-LIGHTS SNOWMEN', '6 RIBBONS ELEGANT CHRISTMAS']
